# Hyperparameter tuning with scikit-learn

This notebooks contains a few examples on how hyperparameter tuning works with scikit-learn.

Author: Umberto Michelucci (umberto.michelucci@toelt.ai).

In [4]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV, GridSearchCV
import pandas as pd

First of all we always need to define what hyperparameters we want to test.

In [5]:
param_grid = {'max_depth': [3, 5, 10], 'min_samples_split': [2, 5, 10]}

Then we need to define what kind of model we want to test.

In [6]:
base_estimator = RandomForestClassifier(random_state=0)

The following cell will generate some *fake* data to use for our tuning.

The `make_classification` function is a utility provided by scikit-learn. This function is specifically used for generating a random n-class classification problem. It is an essential tool for creating synthetic datasets, which are particularly useful for testing and benchmarking machine learning algorithms.

Here are the key aspects of the `make_classification` function:

1. **Purpose**: The primary purpose of `make_classification` is to generate a random, multiclass classification problem. This helps in creating controlled datasets to test classification algorithms and to understand their behavior under various scenarios.

2. **Customizability**: It offers a high degree of customizability. Users can specify several parameters such as the number of samples, the number of features (total, informative, redundant, and repeated), the number of classes, and the level of class separation. This flexibility allows for the creation of datasets with specific characteristics, tailored to particular testing needs.

3. **Output**: The function outputs a tuple containing two arrays: the first array is the generated features (X), and the second array is the class labels for each feature vector (y).

4. **Parameters**:
   - `n_samples`: The number of samples to generate.
   - `n_features`: The total number of features. These include informative, redundant, and noise features.
   - `n_informative`: The number of informative features, i.e., features actually used to build the classification model.
   - `n_redundant`: The number of redundant features, which are linear combinations of the informative features.
   - `n_classes`: The number of classes (or labels) in the dataset.
   - `weights`: The proportions of samples assigned to each class.
   - `class_sep`: A parameter that controls the degree of class separation.
   - And many more parameters to control various aspects of the dataset.


5. **Applications**: It is widely used in machine learning for algorithm development, testing, and comparison. Researchers and practitioners often use it to simulate various data distributions and imbalances to evaluate the performance of classification algorithms under controlled conditions.

6. **Ease of Use**: Like many scikit-learn functions, `make_classification` is user-friendly, making it accessible for both beginners and experienced practitioners in the field of machine learning.

In [7]:
X, y = make_classification(n_samples=1000, random_state=0)

## Hyperparameter Search - Halving Grid Search

`HalvingGridSearchCV` is a function provided by scikit-learn, a widely used Python library for machine learning. It is part of the model selection module and offers an efficient approach to hyperparameter tuning. This function is designed to find the best parameters for a given model through a process known as "successive halving," which is a more resource-efficient version of the traditional grid search.

Key characteristics and functionalities of `HalvingGridSearchCV` include:

1. **Purpose**: The primary objective of `HalvingGridSearchCV` is to identify the best hyperparameters for a given model. It does this by systematically working through multiple combinations of parameter values, evaluating each combination's performance.

2. **Successive Halving Algorithm**: Unlike the traditional grid search that evaluates all combinations of parameter values, `HalvingGridSearchCV` employs the successive halving algorithm. This algorithm initially evaluates a large number of hyperparameter combinations with a small number of resources and then successively halves the number of combinations, allocating more resources to the more promising ones.

3. **Resource Efficiency**: This approach significantly reduces computation time and resource usage, making it more efficient, especially when dealing with large datasets or complex models.

4. **Parameters and Attributes**:
   - It inherits parameters similar to `GridSearchCV`, like `estimator` for the model, `param_grid` for the grid of parameters, `scoring` for the scoring method, and `cv` for cross-validation strategy.
   - `factor`: Determines the rate at which the number of configurations is reduced at each iteration.
   - `resource`: The resource that is increased at each iteration (e.g., the number of iterations for an iterative algorithm).
   - `min_resources`: The minimum amount of resource allocated to each configuration during the first iteration.

5. **Cross-Validation**: It uses cross-validation to evaluate the performance of each set of parameters, ensuring a thorough and unbiased assessment.

6. **Applications**: `HalvingGridSearchCV` is particularly useful in scenarios where the parameter space is large, and a traditional grid search would be too time-consuming or computationally expensive.

7. **Results**: It provides detailed results that include the best parameters found, the score of the best parameters, and the complete results of the search process.


Finally we can do the actual search. By default, the resource is defined in terms of number of samples. That is, each iteration will use an increasing amount of samples to train on. You can however manually specify a parameter to use as the resource with the resource parameter. Here is an example where the resource is defined in terms of the number of estimators of a random forest:

In [5]:
sh = HalvingGridSearchCV(base_estimator, param_grid, cv=5,
                    factor=2, resource='n_estimators',
                    max_resources=30).fit(X, y)

And now we can get the best parameters.

In [6]:
sh.best_estimator_

RandomForestClassifier(max_depth=5, n_estimators=24, random_state=0)

As mentioned above, the number of resources that is used at each iteration depends on the ```min_resources``` parameter. If you have a lot of resources available but start with a low number of resources, some of them might be wasted (i.e. not used). Let us try a different example.

In [8]:
param_grid= {'kernel': ('linear', 'rbf'),
              'C': [1, 10, 100]}
base_estimator = SVC(gamma='scale')

In [9]:
X, y = make_classification(n_samples=1000)

In [12]:
sh = HalvingGridSearchCV(base_estimator, param_grid, cv=5,
                          factor=2, min_resources=20).fit(X, y)

In [13]:
sh.n_resources_

[20, 40, 80]

The search process will only use 80 resources at most, while our maximum amount of available resources is ```n_samples=1000```. Here, we have ```min_resources = r_0 = 20```. For ```HalvingGridSearchCV```, by default, the min_resources parameter is set to ```exhaust```. This means that min_resources is automatically set such that the last iteration can use as many resources as possible, within the max_resources limit

There are many more possibilities, and looking at the official documentation is always a good idea to explore all possibilities.

## Analysis of the results

The cv_results_ attribute contains useful information for analyzing the results of a search. It can be converted to a pandas dataframe with ```df = pd.DataFrame(est.cv_results_)```.

The `cv_results_` attribute in scikit-learn, particularly in model selection tools like `GridSearchCV` and `HalvingGridSearchCV`, is a dictionary that holds a lot of detailed information about the results of the cross-validation process. It is generated after the fitting process (`fit` method) of the model selection tool is completed.

Key aspects of the `cv_results_` attribute include:

1. **Detailed Results**: It contains detailed outcomes for each parameter combination that was evaluated during the cross-validation. This information is invaluable for analyzing and understanding the performance of different hyperparameter configurations.

2. **Structure**: The dictionary includes various keys, each corresponding to different aspects of the results. Common keys include:
   - `mean_test_score`: The mean score of the test set on different folds.
   - `std_test_score`: The standard deviation of the test set score over different folds.
   - `mean_train_score` and `std_train_score`: Similar metrics for the training set (if `return_train_score` is set to True).
   - `params`: A list of parameter settings corresponding to each result.
   - `rank_test_score`: The rank of each parameter setting based on the test score.
   - `split_i_test_score` (where i is the fold number): The test score for each split.
   - `time_fit`: The time taken to fit the model on the train set for each parameter setting.
   - `time_score`: The time taken to score the model on the test set for each parameter setting.

3. **Analysis and Visualization**: This attribute is often used for analyzing the results in detail. You can sort and filter the results to identify the best performing parameter combinations. It is also commonly used for visualizing the results, such as plotting the scores against different hyperparameters to see how they affect the model performance.

4. **Decision Making**: The information in `cv_results_` is critical for making informed decisions about which hyperparameters are the most effective for a given model and dataset.

5. **Post-Processing**: Researchers and practitioners can export this data into data frames (e.g., using Pandas) for easier manipulation and more advanced analyses.


In [17]:
df = pd.DataFrame(sh.cv_results_)
df

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0,20,0.000358,0.000119,0.000158,0.000025,1,linear,"{'C': 1, 'kernel': 'linear'}",0.7500,...,0.6500,0.374166,6,1.0,1.0,1.0,1.0,1.0,1.0,0.0
1,0,20,0.000260,0.000005,0.000140,0.000005,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.7500,...,0.6000,0.300000,9,1.0,1.0,1.0,1.0,1.0,1.0,0.0
2,0,20,0.000361,0.000130,0.000155,0.000023,10,linear,"{'C': 10, 'kernel': 'linear'}",0.7500,...,0.6500,0.374166,6,1.0,1.0,1.0,1.0,1.0,1.0,0.0
3,0,20,0.000521,0.000169,0.000246,0.000058,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.7500,...,0.6000,0.339116,9,1.0,1.0,1.0,1.0,1.0,1.0,0.0
4,0,20,0.000355,0.000063,0.000205,0.000092,100,linear,"{'C': 100, 'kernel': 'linear'}",0.7500,...,0.6500,0.374166,6,1.0,1.0,1.0,1.0,1.0,1.0,0.0
5,0,20,0.000278,0.000014,0.000145,0.000005,100,rbf,"{'C': 100, 'kernel': 'rbf'}",0.7500,...,0.6000,0.339116,9,1.0,1.0,1.0,1.0,1.0,1.0,0.0
6,1,40,0.000345,0.000024,0.000176,0.000019,1,linear,"{'C': 1, 'kernel': 'linear'}",0.8750,...,0.8250,0.100000,2,1.0,1.0,1.0,1.0,1.0,1.0,0.0
7,1,40,0.000604,0.000265,0.000232,0.000092,10,linear,"{'C': 10, 'kernel': 'linear'}",0.8750,...,0.8250,0.100000,2,1.0,1.0,1.0,1.0,1.0,1.0,0.0
8,1,40,0.000308,0.000018,0.000146,0.000005,100,linear,"{'C': 100, 'kernel': 'linear'}",0.8750,...,0.8250,0.100000,2,1.0,1.0,1.0,1.0,1.0,1.0,0.0
9,2,80,0.000452,0.000044,0.000262,0.000108,10,linear,"{'C': 10, 'kernel': 'linear'}",1.0000,...,0.8375,0.101550,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0


Each row corresponds to a given parameter combination (a candidate) and a given iteration. The iteration is given by the ```iter column```. The ```n_resources``` column tells you how many resources were used.

In case you are interested in knowing how hyperband works, you can refer to L. Li, K. Jamieson, G. DeSalvo, A. Rostamizadeh, A. Talwalkar, Hyperband: A Novel Bandit-Based Approach to Hyperparameter Optimization, in Machine Learning Research 18, 2018.

# Classical Grid Search

In [11]:
sh = GridSearchCV(estimator=base_estimator, param_grid=param_grid, cv=5).fit(X, y)

In [12]:
df = pd.DataFrame(sh.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.010620,0.002191,0.000989,0.000178,1,linear,"{'C': 1, 'kernel': 'linear'}",0.930,0.965,0.930,0.885,0.940,0.930,0.025884,1
1,0.008536,0.000126,0.003256,0.000122,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.910,0.950,0.935,0.890,0.925,0.922,0.020640,4
2,0.038135,0.004567,0.000855,0.000046,10,linear,"{'C': 10, 'kernel': 'linear'}",0.930,0.965,0.935,0.875,0.940,0.929,0.029563,2
3,0.008928,0.000406,0.002931,0.000070,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.905,0.910,0.915,0.860,0.895,0.897,0.019647,5
4,0.344359,0.101205,0.000902,0.000051,100,linear,"{'C': 100, 'kernel': 'linear'}",0.930,0.965,0.935,0.875,0.940,0.929,0.029563,2
5,0.009231,0.000503,0.002938,0.000138,100,rbf,"{'C': 100, 'kernel': 'rbf'}",0.895,0.920,0.900,0.860,0.875,0.890,0.020736,6


In [14]:
sh.best_estimator_

SVC(C=1, kernel='linear')

# Appendix A - the Halving Grid Search Algorithm

The Halving Grid Search algorithm, implemented as `HalvingGridSearchCV` in scikit-learn, is an efficient method for hyperparameter tuning. It is a part of the broader family of algorithms known as Successive Halving algorithms. The key idea behind this approach is to iteratively refine the search for the best hyperparameters by allocating resources more efficiently. Here's a detailed description of how it works:

1. **Initial Setup**:
   - The algorithm begins with a predefined grid of hyperparameter values. Each combination of hyperparameters in this grid is considered a candidate.
   - It also requires the definition of a resource, which is usually a measure of how much training a model receives (like the number of iterations, depth of a tree, etc.).

2. **Initial Evaluation**:
   - In the first iteration, all hyperparameter candidates are trained with a small, equal amount of the resource. This initial resource level is usually set by the user.
   - Once trained, each candidate is evaluated using a specified metric (like accuracy, F1-score, etc.).

3. **Selection and Halving**:
   - After evaluating all candidates, a subset of the best-performing candidates is selected. Typically, this selection cuts the number of candidates in half, hence the term "halving."
   - The key aspect of this step is that only a fraction of candidates (the most promising ones) continue to the next round.

4. **Resource Doubling**:
   - For the next iteration, the amount of resource allocated to each remaining candidate is doubled (or increased by a factor specified by the user).
   - These candidates are then re-trained with this increased resource and re-evaluated.

5. **Iterative Process**:
   - Steps 3 and 4 are repeated. In each iteration, the number of candidates is halved, and the resources allocated to each remaining candidate are increased (typically doubled).
   - This process continues until a stopping criterion is met, which could be a maximum resource limit or a minimum number of candidates remaining.

6. **Final Selection**:
   - The final set of hyperparameters is chosen based on the best performance observed in the last iteration of the process.

### Advantages of the Halving Grid Search Algorithm:

- **Efficiency**: By allocating resources more effectively and focusing on promising candidates, it significantly reduces the computational cost compared to traditional grid search, which is beneficial for large hyperparameter spaces.
- **Flexibility**: It works with any type of model and is applicable to a wide range of hyperparameter optimization problems.
- **Scalability**: The algorithm scales well with the size of the dataset and the complexity of the model.

### Key Considerations:

- **Resource Definition**: The choice of resource and how it impacts model training is crucial. The resource could be the number of iterations, number of trees (in ensemble methods), or any other measure that directly affects the training extent of the model.
- **Initial Resource Allocation**: The amount of resource allocated in the initial step can impact the algorithm's effectiveness. Too little resource might not be sufficient to reveal the potential of certain candidates.
